In [5]:
%%writefile new_leak_sol.py

import os
import numpy as np
import pandas as pd
import pickle
import csv
import time
import sys
import math
import random
from collections import defaultdict
import json
import shutil
import bisect
from IPython.display import display, clear_output

nrows_train = 37670294
nrows_test = 2528244
nuser_ids = 1198787
nbookings_train = 3000694

with open('data/train_auto_augmented.csv') as train_file:
    leggo = csv.reader(train_file)
  
    colonne = next(leggo)
    


my_list = list(zip(colonne, range(len(colonne))))

FEAT_DICT = dict(my_list)
LEAK_FEATS = ('id', 'user_location_city', 'srch_destination_id', 'orig_destination_distance')

def get_magnitude(number):
    """
    A function that returns the order of magnitude of a number.
    unit is the label of the order of magnitude, etodivide the exponent
    to divide the number by in order to obtain the appropriate division.
    """
    magnitude = math.log(number, 10)
    n_unit = int(magnitude / 3)
    if n_unit == 0:
        unit = ''
        etodivide = 0
    elif n_unit == 1:
        unit = 'K'
        etodivide = 3
    elif n_unit == 2:
        unit = 'M'
        etodivide = 6
    elif n_unit == 3:
        unit = 'G'
        etodivide = 9
    else:
        unit = 'E' + str(magnitude)
        etodivide = magnitude
        
    return unit, etodivide


def print_processing_status(current_line, tot_line, s_time, frequency=None, pre_message=None):
    """
    A function that prints to screen the processing status of another function.
    Current_line is the line being processed right now, 
    tot_line the total number of lines to be processed,
    s_time the time the computation started,
    frequency how often the print message should be printed (if None, 1/100 of tot_line),
    pre_message the string that should be printed pefore the status message.
    """
    if frequency is None:
        frequency = int(tot_line / 100)
    message = 'Processing line {0:.1f}{4} (of {1:.1f}{4}). \t Elapsed time: {2:.0f} secs, \t ETA: {3:.0f} secs.' 
    
        
    if current_line == 1:
        print('Processing the first line...')
    elif current_line > 1: 
        if current_line % frequency == 0:
            unit, etodivide = get_magnitude(tot_line)
            loc_time = time.time() - s_time

            clear_output(wait=True)
            if pre_message is not None:
                print(pre_message)
            print(message.format(current_line / (10 ** etodivide), tot_line / (10 ** etodivide), 
                  loc_time, (tot_line / current_line - 1) * loc_time, unit))
            sys.stdout.flush()

def encode_key(values):
    try: 
        ret = '+'.join([str(v) for v in values])
    except TypeError:
        ret = str(values)
        
    return ret

def decode_key(key):
    helper = tuple(key.split('+'))
    ret = ()
    for k in helper:
        try: 
            ret += (float(k),)
        except ValueError:
            ret += (k,)
    return ret


def try_float(e):
    try:
        ret = float(e)
    except ValueError:
        ret = e
    return ret


def dd():
    return []


def find_leaked_guess():
    print('Generating keys...')
    
    with open('data/leak_keys_no_nan.pickle', 'rb') as infile:
        leak_vals = pickle.load(infile)

    #sorting the keys, for quicker search of existing keys
    helper = [10**16 * el[1] + 10**8 * el[2] + el[3] for el in leak_vals]
    helper = sorted(helper)
    print('Done!')
    
    clusters = defaultdict(dd)
    guess_count = defaultdict(dd)
    guesses = defaultdict(dd)
    
    #to compute with few memory, it breaks the computation into steps of length CHUNKSIZE
    #and save them to file. n_tmp counts the number of intermediate steps
    CHUNKSIZE = 10**7
    n_tmp = 0
    
    with open('data/train_auto_augmented.csv') as train_file:
        leggo = csv.reader(train_file)
        premessa = 'Reading train lines...'
        print(premessa)
        
        next(leggo)
        
        start_time = time.time()
        for i, line in enumerate(leggo):
            if i>2: print_processing_status(i, nrows_train, start_time, 
                                            frequency=500000, pre_message=premessa)
        
            now_key = tuple(try_float(line[FEAT_DICT[f]]) for f in LEAK_FEATS[1:])
            
            try:
                now_helper = 10**16 * now_key[0] + 10**8 * now_key[1] + now_key[2]
            except TypeError:
                now_helper = -1

            now_key = encode_key(now_key)
            
            #finds the points in helper closest to now_helper
            pos = bisect.bisect_left(helper, now_helper)
            
            #check if the key is present in the saved ones, and if so saves the cluster
            try:
                if now_helper == helper[pos] or now_helper == helper[pos+1]:
                    found = False
                    if len(clusters[now_key]) > 0:
                        for p in clusters[now_key]:
                            if line[FEAT_DICT['hotel_cluster']] == p[0]:
                                p[1] += 1 + (1/(4 * 10**7 - i))
                                found = True                        
                    if not found:
                        clusters[now_key].insert(0, [line[FEAT_DICT['hotel_cluster']], 1])
            except IndexError:
                try:
                    if now_helper == helper[pos]:
                        found = False
                        if len(clusters[now_key]) > 0:
                            for p in clusters[now_key]:
                                if line[FEAT_DICT['hotel_cluster']] == p[0]:
                                    p[1] += 1
                                    found = True                        
                        if not found:
                            clusters[now_key].insert(0, [line[FEAT_DICT['hotel_cluster']], 1 + (1/(4 * 10**7 - i))])
                except IndexError:
                    pass
                
            #saving to file the intermediate step
            if i % CHUNKSIZE == 0 and i>0:
                print('Saving partial result...')
                n_tmp += 1
                tmp_filename = 'data/leak/tmp_' + str(n_tmp) + '.pickle'
                with open(tmp_filename, 'wb') as outftmp:
                    pickle.dump(clusters, outftmp)
                del clusters
                print('Done!')
                clusters = defaultdict(dd)
                
                
        #saving to file the last step
        n_tmp += 1
        tmp_filename = 'data/leak/tmp_' + str(n_tmp) + '.pickle'
        with open(tmp_filename, 'wb') as outftmp:
            pickle.dump(clusters, outftmp)
        del clusters
    

    for c_tmp in range(n_tmp):
        print('Loading partial result...')
        load_tmp_filename = 'data/leak/tmp_' + str(c_tmp+1) + '.pickle'
        with open(load_tmp_filename, 'rb') as inftmp:
            loc_clusters = pickle.load(inftmp)
        print('Done!')

        start_time = time.time()

        for c, el in enumerate(leak_vals):
            print_processing_status(c, 1700000, start_time, frequency=100000, 
                                    pre_message='Converting...(part {} of {})'.format(c_tmp+1, n_tmp))
        
            loc_guesses = loc_clusters[encode_key((el[1], el[2], el[3]))]

            for lg in loc_guesses:
                found = False
                for g in guess_count[el[0]]:
                    if g[0] == lg[0]:
                        g[1] += lg[1]
                        found = True
                if not found:
                    guess_count[el[0]].insert(0, lg)     
                        
        del loc_clusters
        
    print('Converting guess_count to guesses...')
    
    for k in guess_count.keys():
        sortino = sorted(guess_count[k], key=lambda pair: pair[1], reverse=True)
        guesses[k] = [p[0] for p in sortino[:5]]
        
    print('Done!')
        
    return guesses

leak_guesses = find_leaked_guess()

print('Saving clusters...')
with open('data/leak/leak_guesses2.pickle', 'wb') as outf:
    pickle.dump(leak_guesses, outf, -1)
print('Done!!')
print('END')

Writing new_leak_sol.py


In [6]:
leak_guesses

defaultdict(<function __main__.dd>,
            {0.0: [],
             1.0: [],
             2.0: [],
             3.0: ['1'],
             4.0: [['51', 2], ['50', 1], '50'],
             5.0: [],
             6.0: [],
             7.0: [],
             8.0: [['88', 2], '88'],
             9.0: [],
             10.0: [],
             25.0: [['98', 28],
              ['95', 9],
              ['95', 4],
              ['98', 15],
              ['95', 11],
              '95',
              '98'],
             26.0: [],
             27.0: ['39'],
             28.0: [],
             29.0: [['71', 2], ['71', 3], '71'],
             30.0: [],
             32.0: [],
             33.0: [],
             34.0: [['41', 1], '41'],
             35.0: [],
             36.0: [],
             37.0: ['51'],
             38.0: [],
             40.0: [['63', 1], '26'],
             41.0: [['59', 1], ['59', 2], '59'],
             42.0: [],
             43.0: [['79', 2], ['79', 3], '79'],
             44.0: